# Overview

This notebook is organized as follows:
- Section 01 contains the cells responsible for testing the CNN model for CSI compression.
- Section 02 contains the cells responsible for training the CNN model for CSI compression. 

# Section 01

In [1]:
# Import necessary modules
import math
import time
import numpy as np
import scipy.io as sio
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, BatchNormalization, Reshape, Conv2D, add, LeakyReLU
from keras.models import Model, model_from_json
from keras.callbacks import TensorBoard, Callback
tf.compat.v1.reset_default_graph()

In [2]:
# Set image and network params
envir = 'indoor' #choose between 'indoor' or 'outdoor'

# Image params
img_height = 32
img_width = 32
img_channels = 2 
img_total = img_height*img_width*img_channels

# Network params
residual_num = 1 #original value = 2
#compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, 
#compress rate=1/32->dim.=64, compress rate=1/64->dim.=32
encoded_dim = 32 #original value = 512

                    

In [3]:
# Load pre-trained model and weights
file = 'CsiNet_'+(envir)+'_dim'+str(encoded_dim)

# Load json and create model
outfile = "./files_06_channel/saved_model/model_%s.json"%file
json_file = open(outfile, 'r')
loaded_model_json = json_file.read()
json_file.close()
autoencoder = model_from_json(loaded_model_json)

# Load weights from a pre-trained model
outfile = "./files_06_channel/saved_model/model_%s.h5"%file
autoencoder.load_weights(outfile)

In [4]:
# Load dataset for testing
if envir == 'indoor':
  mat = sio.loadmat('files_06_channel/data/DATA_Htestin.mat')
  x_test = mat['HT'] # array

elif envir == 'outdoor':
  mat = sio.loadmat('files_06_channel/data/DATA_Htestout.mat')
  x_test = mat['HT'] # array

x_test = x_test.astype('float32')
x_test = np.reshape(x_test, (len(x_test), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format

In [5]:
# Testing data
tStart = time.time()
x_hat = autoencoder.predict(x_test)
tEnd = time.time()
print ("It took an average of %f seconds per example to evaluate the model on the test set" % ((tEnd - tStart)/x_test.shape[0]))

625/625 [==============================] - 5s 3ms/step
It took an average of 0.000316 seconds per example to evaluate the model on the test set


In [ ]:
# Calculating the NMSE and rho (correlation)

# Load data with 125 (subcarriers) * 32 (antenna) to calculate rho
if envir == 'indoor':
  mat = sio.loadmat('files_06_channel/data/DATA_HtestFin_all.mat')
  X_test = mat['HF_all']# array

elif envir == 'outdoor':
  mat = sio.loadmat('files_06_channel/data/DATA_HtestFout_all.mat')
  X_test = mat['HF_all']# array

X_test = np.reshape(X_test, (len(X_test), img_height, 125))
x_test_real = np.reshape(x_test[:, 0, :, :], (len(x_test), -1))
x_test_imag = np.reshape(x_test[:, 1, :, :], (len(x_test), -1))
x_test_C = x_test_real-0.5 + 1j*(x_test_imag-0.5)
x_hat_real = np.reshape(x_hat[:, 0, :, :], (len(x_hat), -1))
x_hat_imag = np.reshape(x_hat[:, 1, :, :], (len(x_hat), -1))
x_hat_C = x_hat_real-0.5 + 1j*(x_hat_imag-0.5)
x_hat_F = np.reshape(x_hat_C, (len(x_hat_C), img_height, img_width))
X_hat = np.fft.fft(np.concatenate((x_hat_F, np.zeros((len(x_hat_C), img_height, 257-img_width))), axis=2), axis=2)
X_hat = X_hat[:, :, 0:125]

n1 = np.sqrt(np.sum(np.conj(X_test)*X_test, axis=1))
n1 = n1.astype('float64')
n2 = np.sqrt(np.sum(np.conj(X_hat)*X_hat, axis=1))
n2 = n2.astype('float64')
aa = abs(np.sum(np.conj(X_test)*X_hat, axis=1))
rho = np.mean(aa/(n1*n2), axis=1)
X_hat = np.reshape(X_hat, (len(X_hat), -1))
X_test = np.reshape(X_test, (len(X_test), -1))
power = np.sum(abs(x_test_C)**2, axis=1)
power_d = np.sum(abs(X_hat)**2, axis=1)
mse = np.sum(abs(x_test_C-x_hat_C)**2, axis=1)

In [ ]:
print("In "+envir+" environment")
print("When dimension is", encoded_dim)
print("NMSE is ", 10*math.log10(np.mean(mse/power)))
print("Correlation is ", np.mean(rho))

In [ ]:
# Display the original and reconstructed pseudo-gray plots of the strength of H
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1 )
  x_testplo = abs(x_test[i, 0, :, :]-0.5 + 1j*(x_test[i, 1, :, :]-0.5))
  plt.imshow(np.max(np.max(x_testplo))-x_testplo.T)
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  ax.invert_yaxis()
  
  # display reconstructed
  ax = plt.subplot(2, n, i + 1 + n)
  decoded_imgsplo = abs(x_hat[i, 0, :, :]-0.5
                        + 1j*(x_hat[i, 1, :, :]-0.5))
  plt.imshow(np.max(np.max(decoded_imgsplo))-decoded_imgsplo.T)
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  ax.invert_yaxis()

plt.show()

# Section 02

In [ ]:
# Import necessary modules
import math
import time
import numpy as np
import scipy.io as sio
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Input, Dense, BatchNormalization, Reshape, Conv2D, add, LeakyReLU
from keras.models import Model, model_from_json
from keras.callbacks import TensorBoard, Callback
tf.compat.v1.reset_default_graph()

In [ ]:
# Set image and network params
envir = 'indoor' #'indoor' or 'outdoor'

# Image params
img_height = 32
img_width = 32
img_channels = 2 
img_total = img_height*img_width*img_channels

# Network params
residual_num = 2
encoded_dim = 512  #compress rate=1/4->dim.=512, compress rate=1/16->dim.=128, compress rate=1/32->dim.=64, compress rate=1/64->dim.=32

In [ ]:
# Build the autoencoder model (CNN model) of CsiNet
def residual_network(x, residual_num, encoded_dim):
  def add_common_layers(y):
    y = BatchNormalization()(y)
    y = LeakyReLU()(y)
    
    return y
  
  def residual_block_decoded(y):
    shortcut = y
    y = Conv2D(8, kernel_size=(3, 3), padding='same', data_format='channels_first')(y)
    y = add_common_layers(y)
    
    y = Conv2D(16, kernel_size=(3, 3), padding='same', data_format='channels_first')(y)
    y = add_common_layers(y)
    
    y = Conv2D(2, kernel_size=(3, 3), padding='same', data_format='channels_first')(y)
    y = BatchNormalization()(y)

    y = add([shortcut, y])
    y = LeakyReLU()(y)

    return y
  
  x = Conv2D(2, (3, 3), padding='same', data_format="channels_first")(x)
  x = add_common_layers(x)
  
  x = Reshape((img_total,))(x)
  encoded = Dense(encoded_dim, activation='linear')(x)
  
  x = Dense(img_total, activation='linear')(encoded)
  x = Reshape((img_channels, img_height, img_width,))(x)
  for i in range(residual_num):
    x = residual_block_decoded(x)
  
  x = Conv2D(2, (3, 3), activation='sigmoid', padding='same', data_format="channels_first")(x)
  
  return x

In [ ]:
# Set autoencoder model params and display the network model
image_tensor = Input(shape=(img_channels, img_height, img_width))
network_output = residual_network(image_tensor, residual_num, encoded_dim)
autoencoder = Model(inputs=[image_tensor], outputs=[network_output])
autoencoder.compile(optimizer='adam', loss='mse')
print(autoencoder.summary())

In [ ]:
# Load dataset
if envir == 'indoor':
  mat = sio.loadmat('files_06_channel/data/DATA_Htrainin.mat') 
  x_train = mat['HT'] # array
  mat = sio.loadmat('files_06_channel/data/DATA_Hvalin.mat')
  x_val = mat['HT'] # array
  mat = sio.loadmat('files_06_channel/data/DATA_Htestin.mat')
  x_test = mat['HT'] # array

elif envir == 'outdoor':
  mat = sio.loadmat('files_06_channel/data/DATA_Htrainout.mat') 
  x_train = mat['HT'] # array
  mat = sio.loadmat('files_06_channel/data/DATA_Hvalout.mat')
  x_val = mat['HT'] # array
  mat = sio.loadmat('files_06_channel/data/DATA_Htestout.mat')
  x_test = mat['HT'] # array

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
x_train = np.reshape(x_train, (len(x_train), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_val = np.reshape(x_val, (len(x_val), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), img_channels, img_height, img_width))  # adapt this if using `channels_first` image data format

In [ ]:
# Create a class to save the loss history of the model
class LossHistory(Callback):
  def on_train_begin(self, logs={}):
    self.losses_train = []
    self.losses_val = []
  
  def on_batch_end(self, batch, logs={}):
    self.losses_train.append(logs.get('loss'))    
    
  def on_epoch_end(self, epoch, logs={}):
    self.losses_val.append(logs.get('val_loss'))

history = LossHistory()
file = 'CsiNet_'+(envir)+'_dim'+str(encoded_dim)+time.strftime('_%m_%d')
path = 'files_06_channel/result/TensorBoard_%s' %file

In [ ]:
# Train the model with the specified params and save the train loss and val loss in a csv file
autoencoder.fit(x_train, x_train,
                epochs=10, #original value = 1000
                batch_size=20, #original value = 200
                shuffle=True,
                validation_data=(x_val, x_val),
                callbacks=[history,
                           TensorBoard(log_dir = path)])

filename = 'files_06_channel/result/trainloss_%s.csv'%file
loss_history = np.array(history.losses_train)
np.savetxt(filename, loss_history, delimiter=",")

filename = 'files_06_channel/result/valloss_%s.csv'%file
loss_history = np.array(history.losses_val)
np.savetxt(filename, loss_history, delimiter=",")

In [ ]:
# Testing data
tStart = time.time()
x_hat = autoencoder.predict(x_test)
tEnd = time.time()
print ("It cost %f sec" % ((tEnd - tStart)/x_test.shape[0]))

In [ ]:
# Calculating the NMSE and rho
if envir == 'indoor':
  mat = sio.loadmat('files_06_channel/data/DATA_HtestFin_all.mat')
  X_test = mat['HF_all']# array

elif envir == 'outdoor':
  mat = sio.loadmat('files_06_channel/data/DATA_HtestFout_all.mat')
  X_test = mat['HF_all']# array

X_test = np.reshape(X_test, (len(X_test), img_height, 125))
x_test_real = np.reshape(x_test[:, 0, :, :], (len(x_test), -1))
x_test_imag = np.reshape(x_test[:, 1, :, :], (len(x_test), -1))
x_test_C = x_test_real-0.5 + 1j*(x_test_imag-0.5)
x_hat_real = np.reshape(x_hat[:, 0, :, :], (len(x_hat), -1))
x_hat_imag = np.reshape(x_hat[:, 1, :, :], (len(x_hat), -1))
x_hat_C = x_hat_real-0.5 + 1j*(x_hat_imag-0.5)
x_hat_F = np.reshape(x_hat_C, (len(x_hat_C), img_height, img_width))
X_hat = np.fft.fft(np.concatenate((x_hat_F, np.zeros((len(x_hat_C), img_height, 257-img_width))), axis=2), axis=2)
X_hat = X_hat[:, :, 0:125]

n1 = np.sqrt(np.sum(np.conj(X_test)*X_test, axis=1))
n1 = n1.astype('float64')
n2 = np.sqrt(np.sum(np.conj(X_hat)*X_hat, axis=1))
n2 = n2.astype('float64')
aa = abs(np.sum(np.conj(X_test)*X_hat, axis=1))
rho = np.mean(aa/(n1*n2), axis=1)
X_hat = np.reshape(X_hat, (len(X_hat), -1))
X_test = np.reshape(X_test, (len(X_test), -1))
power = np.sum(abs(x_test_C)**2, axis=1)
power_d = np.sum(abs(X_hat)**2, axis=1)
mse = np.sum(abs(x_test_C-x_hat_C)**2, axis=1)

In [ ]:
print("In "+envir+" environment")
print("When dimension is", encoded_dim)
print("NMSE is ", 10*math.log10(np.mean(mse/power)))
print("Correlation is ", np.mean(rho))

filename = "files_06_channel/result/decoded_%s.csv"%file
x_hat1 = np.reshape(x_hat, (len(x_hat), -1))
np.savetxt(filename, x_hat1, delimiter=",")
filename = "files_06_channel/result/rho_%s.csv"%file
np.savetxt(filename, rho, delimiter=",")

In [ ]:
# Display the original and reconstructed pseudo-gray plots of the strength of H
n = 10
plt.figure(figsize=(20, 4))
for i in range(n):
  # display original
  ax = plt.subplot(2, n, i + 1 )
  x_testplo = abs(x_test[i, 0, :, :]-0.5 + 1j*(x_test[i, 1, :, :]-0.5))
  plt.imshow(np.max(np.max(x_testplo))-x_testplo.T)
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  ax.invert_yaxis()
  
  # display reconstructed
  ax = plt.subplot(2, n, i + 1 + n)
  decoded_imgsplo = abs(x_hat[i, 0, :, :]-0.5
                        + 1j*(x_hat[i, 1, :, :]-0.5))
  plt.imshow(np.max(np.max(decoded_imgsplo))-decoded_imgsplo.T)
  plt.gray()
  ax.get_xaxis().set_visible(False)
  ax.get_yaxis().set_visible(False)
  ax.invert_yaxis()

plt.show()

In [ ]:
# Save the model and the weights of the autoencoder
model_json = autoencoder.to_json()
outfile = "files_06_channel/result/my_model_%s.json"%file
with open(outfile, "w") as json_file:
  json_file.write(model_json)
outfile = "files_06_channel/result/my_model_%s.h5"%file
autoencoder.save_weights(outfile)